# ​Basic Usage Tutorial for DMapS​

## 1. ​​Import Essential Dependencies​​

In [ ]:
from qiskit.circuit import Qubit
from qiskit import QuantumCircuit
from qiskit.transpiler import CouplingMap
from qiskit.transpiler.layout import Layout
from qiskit.converters import circuit_to_dag
from qiskit.converters import circuit_to_dag, dag_to_circuit

from router.car_layout_routing import CARSwap
from mapper.two_phase_mapper import TwoPhaseMapper
from frontend.chips_info_reader import QuHardwareInfoReader
from router.multi_mode_routing import (
    MultiModeRouting,
    init_qubit_dist_matrix_v2,
    create_tmp_initial_mapping,
    init_chip_dist_matrix,
)
from router.multi_mode_routing import generate_new_circuit

from global_config import tests_path

## 2. Load/Generate Quantum Circuit​​

In [2]:
qasm_fp = tests_path / "benchmark" / "Bench" / "zcz" / "wstate_n36_transpiled.qasm"
quantum_circuit = QuantumCircuit.from_qasm_file(qasm_fp)

## 3. ​​Construct Target Quantum Chip Network

In [3]:
qcn_fp = tests_path / "hardware_info" / "zcz" / "one_xiaohong_5module_net.json"
chip_type = "zcz"
qcn_obj = QuHardwareInfoReader(qcn_fp)
is_has_multi_chips, qcn = qcn_obj.get_hardware_info(chip_type=chip_type,intra_chip_all2all=True)
qubit_dist_matrix = init_qubit_dist_matrix_v2(qcn)
chip_dist_matrix = init_chip_dist_matrix(qcn)

## 4. Perform End-to-End Initial Qubit Mapping on Target QCN​

In [ ]:
init_mapper = TwoPhaseMapper(fst_phase_iter=50, sec_phase_iter=100)
init_qubit_mapping = init_mapper.run(
    quantum_circuit=quantum_circuit,
    config_fp=qcn_fp,
    chip_type=chip_type,
    dist_matrix=qubit_dist_matrix,
)

num_vers:  36
num_hyEdges:  67
hyperedge_indices:  [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 104, 106, 108, 110, 112, 114, 116, 118, 120, 122, 124, 126, 128, 130, 132, 134, 136, 138, 140, 142, 144, 146, 148, 150, 152, 154, 156, 158, 160, 162, 164, 166, 168]
hyperedges:  [0, 1, 2, 0, 1, 3, 0, 3, 4, 3, 4, 5, 4, 5, 6, 5, 6, 7, 6, 7, 8, 7, 8, 9, 8, 9, 10, 9, 10, 11, 10, 11, 12, 11, 12, 13, 12, 13, 14, 13, 14, 15, 14, 15, 16, 15, 16, 17, 16, 17, 18, 17, 18, 19, 18, 19, 20, 19, 20, 21, 20, 21, 22, 21, 22, 23, 22, 23, 24, 23, 24, 25, 24, 25, 26, 25, 26, 27, 26, 27, 28, 27, 28, 29, 28, 29, 30, 29, 30, 31, 30, 31, 32, 31, 32, 33, 32, 33, 34, 33, 34, 35, 34, 35, 33, 34, 32, 33, 31, 32, 30, 31, 29, 30, 28, 29, 27, 28, 26, 27, 25, 26, 24, 25, 23, 24, 22, 23, 21, 22, 20, 21, 19, 20, 18, 19, 17, 18, 16, 17, 15, 16, 14, 15, 13, 14, 12, 13, 11, 12, 10, 11, 9, 10, 8, 9, 7, 8, 6, 7, 5, 6, 4, 5, 3, 4, 0, 3]
no

In [6]:
init_qubit_mapping

{Qubit(QuantumRegister(36, 'q'), 16): 18,
 Qubit(QuantumRegister(36, 'q'), 17): 15,
 Qubit(QuantumRegister(36, 'q'), 24): 24,
 Qubit(QuantumRegister(36, 'q'), 13): 13,
 Qubit(QuantumRegister(36, 'q'), 12): 12,
 Qubit(QuantumRegister(36, 'q'), 15): 14,
 Qubit(QuantumRegister(36, 'q'), 20): 17,
 Qubit(QuantumRegister(36, 'q'), 19): 16,
 Qubit(QuantumRegister(36, 'q'), 14): 19,
 Qubit(QuantumRegister(36, 'q'), 21): 22,
 Qubit(QuantumRegister(36, 'q'), 22): 20,
 Qubit(QuantumRegister(36, 'q'), 18): 23,
 Qubit(QuantumRegister(36, 'q'), 23): 21,
 Qubit(QuantumRegister(36, 'q'), 11): 56,
 Qubit(QuantumRegister(36, 'q'), 5): 51,
 Qubit(QuantumRegister(36, 'q'), 1): 49,
 Qubit(QuantumRegister(36, 'q'), 6): 52,
 Qubit(QuantumRegister(36, 'q'), 0): 59,
 Qubit(QuantumRegister(36, 'q'), 7): 58,
 Qubit(QuantumRegister(36, 'q'), 2): 48,
 Qubit(QuantumRegister(36, 'q'), 9): 54,
 Qubit(QuantumRegister(36, 'q'), 4): 55,
 Qubit(QuantumRegister(36, 'q'), 8): 57,
 Qubit(QuantumRegister(36, 'q'), 3): 50,
 Q

## 5. ​​Perform End-to-End Qubit Routing

In [7]:
#  Obtain the physical coupling topology information.
couplings_info = []
if is_has_multi_chips:
    chips = qcn.chips
    chip_connections = qcn.chip_connections
    num_phy_qubits = len(qcn.get_total_qubits())

    for chip in chips:
        couplings = chip.couplings
        for coupling in couplings:
            phy_qubit_idx_pair = [coupling[0], coupling[1]]
            couplings_info.append(phy_qubit_idx_pair)

    # Add the information of remote physical connection.
    for chip_connection in chip_connections:
        qubit_pair = chip_connection.qubit_pair
        qubit_idx_pair = [qubit_pair[0].index, qubit_pair[1].index]
        couplings_info.append(qubit_idx_pair)
coupling_map = CouplingMap(couplings_info) 


# Generate the initial layout
tmp_data = {}
init_layout = Layout()
for k, v in init_qubit_mapping.items():
    tmp_data[k.index] = v
tmp_init_mapping = create_tmp_initial_mapping(tmp_data, num_phy_qubits)
_, new_qu_circ = generate_new_circuit(quantum_circuit, num_phy_qubits)
qu_dag = circuit_to_dag(new_qu_circ)
dag_registers = qu_dag.qregs
if len(dag_registers) == 1:
    input_dict = {}
    canonical_register = None
    for _, reg in dag_registers.items():
        canonical_register = reg
    for k, v in tmp_init_mapping.items():
        input_dict[Qubit(register=canonical_register, index=k)] = v
    init_layout.from_dict(input_dict=input_dict)

# ​​Convert distance matrix to list​
qubit_dist_matrix_tolist = qubit_dist_matrix.tolist()
chip_dist_matrix_tolist = chip_dist_matrix.tolist()
router = CARSwap(
    coupling_map=coupling_map,
    chips_net=qcn,
    cost_matrix_tolist=qubit_dist_matrix_tolist,
    chip_dist_matrix_tolist=chip_dist_matrix_tolist,
    heuristic="lookahead",
    fake_run=False,
    input_layout=init_layout,
)
final_dag = router.run(qu_dag)
final_circuit = dag_to_circuit(final_dag)



/tmp/ipykernel_162579/2042575004.py:26: DeprecationWarning: Back-references to from Bit instances to their containing Registers have been deprecated. Instead, inspect Registers to find their contained Bits.
  tmp_data[k.index] = v


# ​​Integrated Qubit Mapping and Routing Algorithm

In [8]:
dqc_compiler = MultiModeRouting()
_, _, mapped_circuit =dqc_compiler.cpa_map_car_rout(
    origin_qasm_fn=qasm_fp,
    chip_info_fn=qcn_fp,
    chip_type=chip_type,
)

num_vers:  36
num_hyEdges:  67
hyperedge_indices:  [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 104, 106, 108, 110, 112, 114, 116, 118, 120, 122, 124, 126, 128, 130, 132, 134, 136, 138, 140, 142, 144, 146, 148, 150, 152, 154, 156, 158, 160, 162, 164, 166, 168]
hyperedges:  [0, 1, 2, 0, 1, 3, 0, 3, 4, 3, 4, 5, 4, 5, 6, 5, 6, 7, 6, 7, 8, 7, 8, 9, 8, 9, 10, 9, 10, 11, 10, 11, 12, 11, 12, 13, 12, 13, 14, 13, 14, 15, 14, 15, 16, 15, 16, 17, 16, 17, 18, 17, 18, 19, 18, 19, 20, 19, 20, 21, 20, 21, 22, 21, 22, 23, 22, 23, 24, 23, 24, 25, 24, 25, 26, 25, 26, 27, 26, 27, 28, 27, 28, 29, 28, 29, 30, 29, 30, 31, 30, 31, 32, 31, 32, 33, 32, 33, 34, 33, 34, 35, 34, 35, 33, 34, 32, 33, 31, 32, 30, 31, 29, 30, 28, 29, 27, 28, 26, 27, 25, 26, 24, 25, 23, 24, 22, 23, 21, 22, 20, 21, 19, 20, 18, 19, 17, 18, 16, 17, 15, 16, 14, 15, 13, 14, 12, 13, 11, 12, 10, 11, 9, 10, 8, 9, 7, 8, 6, 7, 5, 6, 4, 5, 3, 4, 0, 3]
no